In [14]:
import pandas as pd
from mosaic_sql.crate import get_engine, EngineInstance
from sqlalchemy.orm import sessionmaker
from mosaic_sql.postgres import get_engine as get_postgres_engine

In [3]:
%env CRATE_HOST ttda.cratedb-dev-cluster.mosaic.hartreepartners.com:4200
engine = get_engine(EngineInstance.TTDA)
session_mkr = sessionmaker(
    bind=engine, autoflush=False, expire_on_commit=False
)
session = session_mkr()


env: CRATE_HOST=ttda.cratedb-dev-cluster.mosaic.hartreepartners.com:4200


In [13]:
df = pd.read_sql("select * from settles.trader_curves", con=engine)


In [15]:
# postgres_engine = get_postgres_engine('localhost','5432','postgres','docker','postgres')
postgres_engine = get_postgres_engine('ttda.postgres.storage.dev.mosaic.hartreepartners.com',
                                      '5437','postgres','p0stgresisforttda','postgres')

postgres_prod_engine = get_postgres_engine('ttda.postgres.storage.mosaic.hartreepartners.com',
                                      '5437','postgres','p0stgrespr0d4ttda','postgres')

In [19]:
df_dev = pd.read_sql("select * from settles.trader_curves", con=postgres_engine)
df_prod = pd.read_sql("select * from settles.trader_curves where date>'2021-06-01'", con=postgres_prod_engine)

In [20]:
import json
df_dev['definition'] = df_dev['definition'].apply(lambda x: json.dumps(x))
df_prod['definition'] = df_prod['definition'].apply(lambda x: json.dumps(x))

In [8]:
# df_dev.to_sql("trader_curves",schema="settles", index=False, con=postgres_prod_engine, if_exists='append')

In [15]:
# df['date'] = df['stamp']
# df.drop(columns='stamp', inplace=True)
import json
df['definition'] = df['definition'].apply(lambda x: json.dumps(x))

In [16]:
df.to_sql('trader_curves', schema='settles', con=postgres_engine, index=False, if_exists='append')




In [17]:
rdf = pd.read_sql("select * from settles.trader_curves", con=postgres_engine)
print(rdf.head())

   symbol        date      source  \
0  NAP-TS  2021-01-25  arotenberg   
1  NAP-TS  2021-01-25     hartree   
2    RB-F  2021-01-25     hartree   
3     NAP  2021-01-25  arotenberg   
4   DATED  2021-01-25     hartree   

                                          definition  epoch_time  
0  {'type': 'TimeSpread', 'marks': [5.0, 6.0, 5.7...  1611599600  
1  {'type': 'TimeSpread', 'marks': [5.0, 6.0, 5.7...  1611656571  
2  {'type': 'Dynamic', 'contracts': ['202101', '2...  1611599764  
3  {'refs': [{'factor': 8.9, 'symbol': 'BRT-S', '...  1611590261  
4  {'refs': [{'factor': 1.0, 'symbol': 'BRT-S'}, ...  1611599766  


In [10]:
row = rdf.loc[0,'definition']

In [23]:
# Copy prod data to dev db
df_prod.to_sql('trader_curves',  schema='settles', con=postgres_engine,  index=False, if_exists='append')